In [3]:
# %%
################################################################################
# Title: Second Score Sensitivity Simulation – Python Implementation
# Author: Tirth Bhatt
#
# Project: [Placeholder] 
#
# Notes:
# - One common θ (simple logit).
# - Closed‑form mark‑up formula taken from the R prototype.
# - Three worlds:
#     1. Pre‑merger date (no merger)
#     2. Post‑merger date (no merger) – entrant’s share grows
#     3. Post‑merger date (with merger) – Firm 1 merges with Firm 2
# - No external files.  Only numpy required.
################################################################################

# %% imports
import numpy as np

# %% --------------------------- INPUTS ----------------------------------------
theta = 0.5

prices = np.array([100, 150,  90, 100, 110], dtype=float)
product_names = np.array(["Product 1", "Product 2", "Product 3",
                          "Product 4", "Product 5"])

owners_pre  = np.array([0, 0, 0, 1, 2])          # Firm 1, Firm 1, Firm 1, Firm 2, Firm 3
owners_post = np.array([0, 0, 0, 0, 2])          # Firm 1+2, Firm 3
firm_names  = np.array(["Firm 1", "Firm 2", "Firm 3"])

# World‑1 inside‑market percentages (must sum to 1)
inside_pct_w1 = np.array([0.50, 0.20, 0.20, 0.10, 1e-9])
outside_w1    = 0.10

# Margin for Product 4 (Firm 2)
margin_p4 = 0.40

# World‑2 percentages after entrant gains share (still sum to 1)
inside_pct_w2 = np.array([0.40, 0.15, 0.15, 0.10, 0.20])
outside_w2    = 0.10
# -----------------------------------------------------------------------------


# %% ---------------------- HELPER FUNCTIONS -----------------------------------
EPS = 1e-12

def pct_to_abs(pct_vec, s0):
    """Inside‑pct → absolute share, clipping zeros to EPS."""
    pct = pct_vec.copy()
    pct[pct < EPS] = EPS
    pct /= pct.sum()
    return (1 - s0) * pct

def alpha_from_reference(price, cost, share, th):
    return ((1 - th) / (price - cost)) * (1 / share) * np.log(1 / (1 - share))

def firm_total(shares, owners, n_firms):
    """Return ∑_j S_j for each firm index 0..n_firms‑1."""
    res = np.zeros(n_firms)
    for s, f in zip(shares, owners):
        res[f] += s
    res[res < EPS] = EPS
    return res

def markups(th, alpha, firm_sh):
    """Markup for each firm (vector length = n_firms)."""
    return ((1 - th) / alpha) * (1 / firm_sh) * np.log(1 / (1 - firm_sh))


# %% ------------------------- WORLD 1  (pre) ----------------------------------
inside_w1 = pct_to_abs(inside_pct_w1, outside_w1)

cost_p4   = (1 - margin_p4) * prices[3]                       # unit cost for p4
alpha     = alpha_from_reference(prices[3], cost_p4, inside_w1[3], theta)

firm_share_w1   = firm_total(inside_w1, owners_pre, 3)
firm_markup_w1  = markups(theta, alpha, firm_share_w1)

costs = prices.copy()
for i, owner in enumerate(owners_pre):
    costs[i] = prices[i] - firm_markup_w1[owner]
costs[3] = cost_p4                                            # overwrite with known cost

# %% ------------------------- WORLD 2  (post, no merger) ---------------------
inside_w2        = pct_to_abs(inside_pct_w2, outside_w2)
firm_share_w2    = firm_total(inside_w2, owners_pre, 3)
firm_markup_w2   = markups(theta, alpha, firm_share_w2)
prices_w2        = costs + firm_markup_w2[owners_pre]

# %% ------------------------- WORLD 3  (post, with merger) -------------------
firm_share_w3    = firm_total(inside_w2, owners_post, 3)      # only two active firms
firm_markup_w3   = markups(theta, alpha, firm_share_w3)
prices_w3        = costs + firm_markup_w3[owners_post]

# %% ------------------------- SUMMARY PRINT ----------------------------------

def banner(txt):
    print("\n" + txt)
    print("-" * len(txt))

# Scenario 1 output
banner("World 1  –  No Entry, No Merger")
print(f"Outside share           : {outside_w1:.4f}")
print(f"Alpha                   : {alpha:.6f}")
print(f"Theta                   : {theta:.2f}\n")

print("By Firm (shares & mark‑ups):")
for f_idx, firm in enumerate(firm_names):
    inside_pct_firm = inside_pct_w1[owners_pre == f_idx].sum()
    print(f"  {firm:<6s} inside‑pct {inside_pct_firm:.4f}  share {firm_share_w1[f_idx]:.4f}  markup {firm_markup_w1[f_idx]:.4f}")
    for i in np.where(owners_pre == f_idx)[0]:
        print(f"     {product_names[i]}  Price {prices[i]:.2f}  Cost {costs[i]:.2f}")

# Scenario 2 output
banner("World 2  –  Entry, No Merger")
for i, name in enumerate(product_names):
    print(f"{name:<10s} Price {prices_w2[i]:.2f}")

# Scenario 3 output
banner("World 3  –  Entry, With merger")
for i, name in enumerate(product_names):
    print(f"{name:<10s} Price {prices_w3[i]:.2f}")

# Price effects
banner("PRICE EFFECT  (Effect of Only the Merger)")
for i, name in enumerate(product_names):
    delta = prices_w3[i] - prices_w2[i]
    pct   = 100 * delta / prices_w2[i]
    print(f"{name:<10s} {pct:.2f}%")



SCENARIO 1  –  Pre‑merger (no merger)
-------------------------------------
Outside share           : 0.1000
Alpha                   : 0.013099
Theta                   : 0.50

By Firm (shares & mark‑ups):
  Firm 1 inside‑pct 0.9000  share 0.8100  markup 78.2629
     Product 1  Price 100.00  Cost 21.74
     Product 2  Price 150.00  Cost 71.74
     Product 3  Price 90.00  Cost 11.74
  Firm 2 inside‑pct 0.1000  share 0.0900  markup 40.0000
     Product 4  Price 100.00  Cost 60.00
  Firm 3 inside‑pct 0.0000  share 0.0000  markup 38.1717
     Product 5  Price 110.00  Cost 71.83

SCENARIO 2  –  Post‑merger date, no merger
------------------------------------------
Product 1  Price 81.98
Product 2  Price 131.98
Product 3  Price 71.98
Product 4  Price 100.00
Product 5  Price 113.91

SCENARIO 3  –  Post‑merger date, with merger
--------------------------------------------
Product 1  Price 89.22
Product 2  Price 139.22
Product 3  Price 79.22
Product 4  Price 127.49
Product 5  Price 113.91

PRIC